In [329]:
import pandas as pd
import numpy as np
import os

In [330]:
os.getcwd()

'/Users/huangyiyang/Desktop/Winter_2025/MACS30122/final-project-ascending(feb 26)/notebooks'

**Merge mobility rates with county and cz data**

Based on cz_id and FIPS

Output: county_mobility_df (df) and cz_mobility_df (df)

In [331]:
current_dir = os.getcwd()

cz_file = "mobility_cz.csv"
county_file = "mobility_county.csv"
county_info_file = "county_info.csv"

cz_file_path = '/Users/huangyiyang/Desktop/Winter_2025/MACS30122/final-project-ascending(feb 26)/mobility_cz.csv'
county_file_path = '/Users/huangyiyang/Desktop/Winter_2025/MACS30122/final-project-ascending(feb 26)/mobility_county.csv'
cz_info_file_path = '/Users/huangyiyang/Desktop/Winter_2025/MACS30122/final-project-ascending(feb 26)/county_info.csv'
school_tract_cz_file_path = '/Users/huangyiyang/Desktop/Winter_2025/MACS30122/final-project-ascending(feb 26)/data/links/school_tract_cz_merged.csv'

mobility_cz = pd.read_csv(cz_file_path)
mobility_county = pd.read_csv(county_file_path)
cz_info = pd.read_csv(cz_info_file_path)
school_tract_cz = pd.read_csv(school_tract_cz_file_path)
# mobility_cz
school_tract_cz.columns

Index(['geometry', 'index_right', 'Tract_FIPS', 'inside_point',
       'geometry_tract', 'distance', 'universal-id', 'State', 'CZ_ID',
       'County_FIPS'],
      dtype='object')

In [332]:

mobility_cz.rename(columns={'AM, 80-82 Cohort': 'Absolute_Upward_Mobility', 'CZ':'CZ_ID'}, inplace=True)


mobility_cz

,CZ_ID,CZ Name,State,Children in 1980-82 Cohorts,Absolute_Upward_Mobility,"RM, 80-82 Cohort","P(Child in Q5 | Parent in Q1), 80-85 Cohort","AM, College Attendance","RM, College Attendance","AM, Teenage birth",...,"AM, 83-85 Cohort","RM, 83-85 Cohort","AM, 80-85 Cohort","RM, 80-85 Cohort","AM, %White>80% ZIPs","RM, %White>80% ZIPs","AM, Poverty Line","RM, Poverty Line","AM, College Quality","RM, College Quality"
0,(1),(2),(3),(4),(5),(6),(7),(8),(9),(10),...,(26),(27),(28),(29),(30),(31),(32),(33),(34),(35)
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100,Johnson City,TN,"18,494",38.4,0.372,0.062,33.7,0.875,23.1,...,40.0,0.349,39.2,0.359,38.4,0.372,66.6,0.402,35.1,0.207
3,200,Morristown,TN,"7,760",37.8,0.356,0.054,32.7,0.775,27.2,...,39.4,0.371,38.6,0.362,37.8,0.356,65.3,0.439,34.7,0.169
4,301,Middlesborough,TN,"2,217",39.0,0.376,0.073,41.0,0.680,23.2,...,40.0,0.352,39.5,0.362,39.1,0.375,65.0,0.483,36.1,0.158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
738,39205,John Day,OR,353,49.7,0.166,0.116,47.0,0.433,NaN,...,47.6,0.353,48.6,0.260,49.6,0.167,81.5,0.129,40.5,0.105
739,39301,Friday Harbor,WA,439,39.1,0.255,0.102,48.7,0.528,NaN,...,43.4,0.123,41.4,0.186,39.1,0.255,67.7,0.236,45.8,0.137
740,39302,Bellingham,WA,"5,809",44.1,0.276,0.116,46.4,0.566,14.7,...,44.6,0.267,44.4,0.271,44.2,0.274,71.5,0.267,39.2,0.152
741,39303,Port Angeles,WA,"3,366",41.4,0.285,0.086,40.3,0.577,19.1,...,41.7,0.253,41.6,0.269,41.8,0.275,70.3,0.293,36.8,0.147


**Merge commuting zone mobility rate to school_tract_cz**

In [333]:
# Strip spaces and ensure formatting is consistent
mobility_cz['CZ_ID'] = mobility_cz['CZ_ID'].astype(str).str.strip()
school_tract_cz['CZ_ID'] = school_tract_cz['CZ_ID'].astype(str).str.strip()

# Debug: Find mismatched CZ_IDs
unmatched_ids = school_tract_cz[~school_tract_cz['CZ_ID'].isin(mobility_cz['CZ_ID'])]
# print("Unmatched CZ_IDs:", unmatched_ids['CZ_ID'].unique())

# If leading zeros are missing, adjust (Modify zfill(5) based on your actual CZ_ID length)
mobility_cz['CZ_ID'] = mobility_cz['CZ_ID'].str.zfill(5)
school_tract_cz['CZ_ID'] = school_tract_cz['CZ_ID'].str.zfill(5)

# Merge
cz_mobility_df = school_tract_cz.merge(
    mobility_cz[['CZ_ID', 'Absolute_Upward_Mobility']], 
    on='CZ_ID', 
    how='left'
)
cz_mobility_df.head(10)

,geometry,index_right,Tract_FIPS,inside_point,geometry_tract,distance,universal-id,State,CZ_ID,County_FIPS,Absolute_Upward_Mobility
0,POINT (1093362.8431272167 2259200.009510496),2650,17031460100,POINT (-87.53609281286779 41.7419725),"POLYGON ((-87.55407199999999 41.751895, -87.55...",421533.046700,2613500,IL,24300,17031,39.4
1,POINT (1090058.261661958 2258171.498672869),2650,17031460100,POINT (-87.53609281286779 41.7419725),"POLYGON ((-87.55407199999999 41.751895, -87.55...",418076.840995,2613517,IL,24300,17031,39.4
2,POINT (1093342.0194390013 2259032.995433893),2650,17031460100,POINT (-87.53609281286779 41.7419725),"POLYGON ((-87.55407199999999 41.751895, -87.55...",421455.671632,2613577,IL,24300,17031,39.4
3,POINT (1090277.6885607098 2258107.9643670265),2650,17031460100,POINT (-87.53609281286779 41.7419725),"POLYGON ((-87.55407199999999 41.751895, -87.55...",418260.625540,2600307,IL,24300,17031,39.4
4,POINT (1090058.261661958 2258171.498672869),2650,17031460100,POINT (-87.53609281286779 41.7419725),"POLYGON ((-87.55407199999999 41.751895, -87.55...",418076.840995,2600311,IL,24300,17031,39.4
5,POINT (1049013.7193265455 2286655.8233918143),2461,17031808702,POINT (-87.676259253533 42.054739999999995),"POLYGON ((-87.681749 42.052076, -87.6817109999...",391429.954627,2600334,IL,24300,17031,39.4
6,POINT (1048617.904624108 2286524.4810885),2461,17031808702,POINT (-87.676259253533 42.054739999999995),"POLYGON ((-87.681749 42.052076, -87.6817109999...",391013.330263,2600335,IL,24300,17031,39.4
7,POINT (1048825.0187941655 2286636.802836942),2461,17031808702,POINT (-87.676259253533 42.054739999999995),"POLYGON ((-87.681749 42.052076, -87.6817109999...",391246.892487,2606038,IL,24300,17031,39.4
8,POINT (1062612.0408869137 2278934.8978108214),2650,17031460100,POINT (-87.53609281286779 41.7419725),"POLYGON ((-87.55407199999999 41.751895, -87.55...",400588.163699,2600394,IL,24300,17031,39.4
9,POINT (1063089.4733588537 2279602.846673257),2650,17031460100,POINT (-87.53609281286779 41.7419725),"POLYGON ((-87.55407199999999 41.751895, -87.55...",401299.439476,2600395,IL,24300,17031,39.4


**Merge county mobility rate to school_tract_cz**

In [334]:
mobility_county.rename(columns={'Absolute Upward Mobility': 'Absolute_Upward_Mobility', 'County FIPS Code': 'County_FIPS'}, inplace=True)
mobility_county = mobility_county.drop(index=0)

mobility_county

,County_FIPS,County Name,Commuting Zone ID,Commuting Zone Name,State,Number of Children in Core Sample,Rank-Rank Slope,Absolute_Upward_Mobility,Top 1% Income Share,Interquartile Income Range,...,Parent Income P25,Child Income P25,Median Parent Income,Median Child Income,Parent Income P75,Child Income P75,Parent Income P90,Child Income P90,Parent Income P99,Child Income P99
1,1001,Autauga,11101,Montgomery,Alabama,"1,846",0.444,38.4,0.061,60882,...,26900,13700,53900,31900,87700,62100,121600,90400,227400,158000
2,1003,Baldwin,11001,Mobile,Alabama,"5,061",0.336,39.6,0.127,63854,...,26600,13400,53600,30200,90500,59500,135600,93900,444500,194500
3,1005,Barbour,10301,Eufaula,Alabama,"1,129",0.398,36.1,0.097,41924,...,18900,10400,32000,22400,60800,44600,105600,75800,235700,153400
4,1007,Bibb,10801,Tuscaloosa,Alabama,841,0.434,39.1,0.066,52496,...,18300,11600,36500,26800,70800,57100,98300,89300,204300,165600
5,1009,Blount,10700,Birmingham,Alabama,"1,224",0.317,42.0,0.063,46598,...,28100,13900,50200,33200,74700,61300,101400,87000,213900,159800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,56037,Sweetwater,36404,Rock Springs,Wyoming,"2,349",0.204,53.1,0.076,48201,...,56500,24700,83900,52800,104700,89700,131100,121600,384700,217000
3138,56039,Teton,36303,Jackson,Wyoming,482,0.145,50.4,0.669,111053,...,49500,21300,87500,44100,160500,76900,342400,114900,3040200,306500
3139,56041,Uinta,36404,Rock Springs,Wyoming,"1,366",0.316,50.3,0.040,48930,...,49200,24000,76800,55000,98100,83700,119800,110600,239200,212800
3140,56043,Washakie,34302,Worland,Wyoming,461,0.342,47.5,0.184,43050,...,38000,19200,61400,45800,81100,76900,107800,107800,261500,193600


In [335]:
# Strip spaces and ensure formatting is consistent
mobility_county['County_FIPS'] = mobility_county['County_FIPS'].astype(str).str.strip()
school_tract_cz['County_FIPS'] = school_tract_cz['County_FIPS'].astype(str).str.strip()

# Debug: Find mismatched CZ_IDs
unmatched_ids = school_tract_cz[~school_tract_cz['County_FIPS'].isin(mobility_county['County_FIPS'])]
print("Unmatched County_FIPS:", unmatched_ids['County_FIPS'].unique())

# If leading zeros are missing, adjust (Modify zfill(5) based on your actual CZ_ID length)
mobility_county['County_FIPS'] = mobility_county['County_FIPS'].str.zfill(5)
school_tract_cz['County_FIPS'] = school_tract_cz['County_FIPS'].str.zfill(5)

# Merge again
county_mobility_df = school_tract_cz.merge(
    mobility_county[['County_FIPS', 'Absolute_Upward_Mobility']], 
    on='County_FIPS', 
    how='left'
)
county_mobility_df.head(10)

Unmatched County_FIPS: []


,geometry,index_right,Tract_FIPS,inside_point,geometry_tract,distance,universal-id,State,CZ_ID,County_FIPS,Absolute_Upward_Mobility
0,POINT (1093362.8431272167 2259200.009510496),2650,17031460100,POINT (-87.53609281286779 41.7419725),"POLYGON ((-87.55407199999999 41.751895, -87.55...",421533.046700,2613500,IL,24300,17031,38.0
1,POINT (1090058.261661958 2258171.498672869),2650,17031460100,POINT (-87.53609281286779 41.7419725),"POLYGON ((-87.55407199999999 41.751895, -87.55...",418076.840995,2613517,IL,24300,17031,38.0
2,POINT (1093342.0194390013 2259032.995433893),2650,17031460100,POINT (-87.53609281286779 41.7419725),"POLYGON ((-87.55407199999999 41.751895, -87.55...",421455.671632,2613577,IL,24300,17031,38.0
3,POINT (1090277.6885607098 2258107.9643670265),2650,17031460100,POINT (-87.53609281286779 41.7419725),"POLYGON ((-87.55407199999999 41.751895, -87.55...",418260.625540,2600307,IL,24300,17031,38.0
4,POINT (1090058.261661958 2258171.498672869),2650,17031460100,POINT (-87.53609281286779 41.7419725),"POLYGON ((-87.55407199999999 41.751895, -87.55...",418076.840995,2600311,IL,24300,17031,38.0
5,POINT (1049013.7193265455 2286655.8233918143),2461,17031808702,POINT (-87.676259253533 42.054739999999995),"POLYGON ((-87.681749 42.052076, -87.6817109999...",391429.954627,2600334,IL,24300,17031,38.0
6,POINT (1048617.904624108 2286524.4810885),2461,17031808702,POINT (-87.676259253533 42.054739999999995),"POLYGON ((-87.681749 42.052076, -87.6817109999...",391013.330263,2600335,IL,24300,17031,38.0
7,POINT (1048825.0187941655 2286636.802836942),2461,17031808702,POINT (-87.676259253533 42.054739999999995),"POLYGON ((-87.681749 42.052076, -87.6817109999...",391246.892487,2606038,IL,24300,17031,38.0
8,POINT (1062612.0408869137 2278934.8978108214),2650,17031460100,POINT (-87.53609281286779 41.7419725),"POLYGON ((-87.55407199999999 41.751895, -87.55...",400588.163699,2600394,IL,24300,17031,38.0
9,POINT (1063089.4733588537 2279602.846673257),2650,17031460100,POINT (-87.53609281286779 41.7419725),"POLYGON ((-87.55407199999999 41.751895, -87.55...",401299.439476,2600395,IL,24300,17031,38.0


In [336]:
print(county_mobility_df['Absolute_Upward_Mobility'].isna().sum())
print(cz_mobility_df['Absolute_Upward_Mobility'].isna().sum())


19
0


In [337]:
county_mobility_df = county_mobility_df.dropna(subset=['Absolute_Upward_Mobility'])
print(county_mobility_df['Absolute_Upward_Mobility'].isna().sum())

0


**Merge Scraped data with sentiment score**

Merge review_data_all_state.csv and scraped_ratings_all_states.csv based on "universal_id"

In [338]:
review_path = '/Users/huangyiyang/Desktop/Winter_2025/MACS30122/final-project-ascending(feb 26)/data/greatschools/review_data_all_states.csv'
school_path = '/Users/huangyiyang/Desktop/Winter_2025/MACS30122/final-project-ascending(feb 26)/data/greatschools/scraped_ratings_all_states.csv'

review_df = pd.read_csv(review_path)
schools_df = pd.read_csv(school_path)

/var/folders/3t/vdscgrwj3jz8r27lrkl5_f_80000gn/T/ipykernel_89375/625743744.py:5: DtypeWarning: Columns (16,18,20,22,26,28,30,32,36,40,72,73,76,78,79,80,81,82,83,84,85,86,87,88,89,90,91,102,104,106,108,110,112,120,121,129,130,131,132,144,145,154,155,163,164,167,168) have mixed types. Specify dtype option on import or set low_memory=False.
  schools_df = pd.read_csv(school_path)


In [339]:
print(review_df.columns)
print(schools_df.columns)

Index(['universal_id', 'overview-url', 'state', 'reviews', 'sentiment_scores'], dtype='object')
Index(['school_id', 'demographics_ethnicity_White',
       'demographics_ethnicity_Two or more races',
       'demographics_ethnicity_Hispanic', 'demographics_ethnicity_Black',
       'demographics_ethnicity_Asian or Pacific Islander',
       'demographics_ethnicity_Native American',
       'demographics_ethnicity_Native Hawaiian or Other Pacific Islander',
       'demographics_ethnicity_Unspecified', 'demographics_gender_Male',
       ...
       'school_improvement_Engaged Families',
       'school_improvement_Supported Students',
       'test_scores_Social Studies_Grade 4_school_value',
       'test_scores_Social Studies_Grade 4_state_value',
       'college_success_Graduates pursuing college_school_value',
       'college_success_Graduates pursuing college_state_value',
       'test_scores_Social Studies_Grade 10_school_value',
       'test_scores_Social Studies_Grade 10_state_value',
   

In [340]:
schools_df.rename(columns={'school_id': 'universal-id'}, inplace=True)
review_df.rename(columns={'universal_id': 'universal-id'}, inplace=True)

In [341]:
schools_df = schools_df.merge(
    review_df[['universal-id', 'sentiment_scores']],
    on='universal-id',
    how='left'
)
schools_df.head(10)

,universal-id,demographics_ethnicity_White,demographics_ethnicity_Two or more races,demographics_ethnicity_Hispanic,demographics_ethnicity_Black,demographics_ethnicity_Asian or Pacific Islander,demographics_ethnicity_Native American,demographics_ethnicity_Native Hawaiian or Other Pacific Islander,demographics_ethnicity_Unspecified,demographics_gender_Male,...,school_improvement_Supported Students,test_scores_Social Studies_Grade 4_school_value,test_scores_Social Studies_Grade 4_state_value,college_success_Graduates pursuing college_school_value,college_success_Graduates pursuing college_state_value,test_scores_Social Studies_Grade 10_school_value,test_scores_Social Studies_Grade 10_state_value,college_success_Graduates pursuing 4-year college_school_value,college_success_Graduates pursuing 4-year college_state_value,sentiment_scores
0,1700001,89.0,1.0,9.0,1.0,0.0,0.0,0.0,NaN,47.407407,...,65.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1700002,90.0,2.0,9.0,0.0,0.0,0.0,0.0,NaN,49.523809,...,42.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1700005,95.0,1.0,2.0,2.0,1.0,0.0,0.0,NaN,52.328767,...,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1700007,95.0,2.0,2.0,1.0,0.0,0.0,0.0,NaN,51.498127,...,69.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1700009,94.0,4.0,2.0,1.0,0.0,0.0,0.0,NaN,51.063829,...,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1700011,92.0,4.0,3.0,0.0,0.0,0.0,0.0,1.0,50.367647,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1700012,91.0,7.0,1.0,0.0,0.0,0.0,0.0,1.0,49.047619,...,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1700013,95.0,2.0,2.0,0.0,0.0,0.0,0.0,1.0,52.898550,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'neg': 0.019, 'neu': 0.737, 'pos': 0.245, 'co..."
8,1700014,96.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,50.246305,...,44.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1700015,100.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,54.878048,...,88.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'neg': 0.074, 'neu': 0.823, 'pos': 0.103, 'co..."


In [342]:
print((schools_df).shape)
print(schools_df['sentiment_scores'].isna().sum())
print(f"Review missingness: {schools_df['sentiment_scores'].isna().sum() / schools_df.shape[0]:.2f}")

(19612, 170)
11093
Review missingness: 0.57


In [343]:

# Function to ensure sentiment_scores contains dictionaries
def clean_sentiment(x):
    if isinstance(x, dict):  # Keep existing valid sentiment dictionaries
        return x
    elif isinstance(x, str):  # If sentiment scores are stored as strings, convert them
        try:
            import ast
            return ast.literal_eval(x)  # Convert string to dictionary if possible
        except:
            return {'neg': 0, 'neu': 0, 'pos': 0, 'compound': 0}
    elif pd.isna(x):  # Fill missing values with zero scores
        return {'neg': 0, 'neu': 0, 'pos': 0, 'compound': 0}
    else:  # For any other unexpected type, default to zero scores
        return {'neg': 0, 'neu': 0, 'pos': 0, 'compound': 0}

# Apply the function to clean sentiment_scores column
schools_df['sentiment_scores'] = schools_df['sentiment_scores'].apply(clean_sentiment)

# Expand the dictionary into separate columns
sentiment_df = schools_df['sentiment_scores'].apply(pd.Series)

# Concatenate the new columns with the original DataFrame
schools_df = pd.concat([schools_df, sentiment_df], axis=1)



In [344]:
schools_df.head(10)

,universal-id,demographics_ethnicity_White,demographics_ethnicity_Two or more races,demographics_ethnicity_Hispanic,demographics_ethnicity_Black,demographics_ethnicity_Asian or Pacific Islander,demographics_ethnicity_Native American,demographics_ethnicity_Native Hawaiian or Other Pacific Islander,demographics_ethnicity_Unspecified,demographics_gender_Male,...,college_success_Graduates pursuing college_state_value,test_scores_Social Studies_Grade 10_school_value,test_scores_Social Studies_Grade 10_state_value,college_success_Graduates pursuing 4-year college_school_value,college_success_Graduates pursuing 4-year college_state_value,sentiment_scores,neg,neu,pos,compound
0,1700001,89.0,1.0,9.0,1.0,0.0,0.0,0.0,NaN,47.407407,...,NaN,NaN,NaN,NaN,NaN,"{'neg': 0, 'neu': 0, 'pos': 0, 'compound': 0}",0.000,0.000,0.000,0.0000
1,1700002,90.0,2.0,9.0,0.0,0.0,0.0,0.0,NaN,49.523809,...,NaN,NaN,NaN,NaN,NaN,"{'neg': 0, 'neu': 0, 'pos': 0, 'compound': 0}",0.000,0.000,0.000,0.0000
2,1700005,95.0,1.0,2.0,2.0,1.0,0.0,0.0,NaN,52.328767,...,NaN,NaN,NaN,NaN,NaN,"{'neg': 0, 'neu': 0, 'pos': 0, 'compound': 0}",0.000,0.000,0.000,0.0000
3,1700007,95.0,2.0,2.0,1.0,0.0,0.0,0.0,NaN,51.498127,...,NaN,NaN,NaN,NaN,NaN,"{'neg': 0, 'neu': 0, 'pos': 0, 'compound': 0}",0.000,0.000,0.000,0.0000
4,1700009,94.0,4.0,2.0,1.0,0.0,0.0,0.0,NaN,51.063829,...,NaN,NaN,NaN,NaN,NaN,"{'neg': 0, 'neu': 0, 'pos': 0, 'compound': 0}",0.000,0.000,0.000,0.0000
5,1700011,92.0,4.0,3.0,0.0,0.0,0.0,0.0,1.0,50.367647,...,NaN,NaN,NaN,NaN,NaN,"{'neg': 0, 'neu': 0, 'pos': 0, 'compound': 0}",0.000,0.000,0.000,0.0000
6,1700012,91.0,7.0,1.0,0.0,0.0,0.0,0.0,1.0,49.047619,...,NaN,NaN,NaN,NaN,NaN,"{'neg': 0, 'neu': 0, 'pos': 0, 'compound': 0}",0.000,0.000,0.000,0.0000
7,1700013,95.0,2.0,2.0,0.0,0.0,0.0,0.0,1.0,52.898550,...,NaN,NaN,NaN,NaN,NaN,"{'neg': 0.019, 'neu': 0.737, 'pos': 0.245, 'co...",0.019,0.737,0.245,0.9945
8,1700014,96.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,50.246305,...,NaN,NaN,NaN,NaN,NaN,"{'neg': 0, 'neu': 0, 'pos': 0, 'compound': 0}",0.000,0.000,0.000,0.0000
9,1700015,100.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,54.878048,...,NaN,NaN,NaN,NaN,NaN,"{'neg': 0.074, 'neu': 0.823, 'pos': 0.103, 'co...",0.074,0.823,0.103,0.9181


**Aggregate to CZ Level**

In [345]:
# add a column for corresponding CZ_ID from school_tract_cz
schools_df = schools_df.merge(
    school_tract_cz[['universal-id', 'CZ_ID']],
    on='universal-id',
    how='left'
)
schools_df.head(10)

,universal-id,demographics_ethnicity_White,demographics_ethnicity_Two or more races,demographics_ethnicity_Hispanic,demographics_ethnicity_Black,demographics_ethnicity_Asian or Pacific Islander,demographics_ethnicity_Native American,demographics_ethnicity_Native Hawaiian or Other Pacific Islander,demographics_ethnicity_Unspecified,demographics_gender_Male,...,test_scores_Social Studies_Grade 10_school_value,test_scores_Social Studies_Grade 10_state_value,college_success_Graduates pursuing 4-year college_school_value,college_success_Graduates pursuing 4-year college_state_value,sentiment_scores,neg,neu,pos,compound,CZ_ID
0,1700001,89.0,1.0,9.0,1.0,0.0,0.0,0.0,NaN,47.407407,...,NaN,NaN,NaN,NaN,"{'neg': 0, 'neu': 0, 'pos': 0, 'compound': 0}",0.000,0.000,0.000,0.0000,23200
1,1700002,90.0,2.0,9.0,0.0,0.0,0.0,0.0,NaN,49.523809,...,NaN,NaN,NaN,NaN,"{'neg': 0, 'neu': 0, 'pos': 0, 'compound': 0}",0.000,0.000,0.000,0.0000,23200
2,1700005,95.0,1.0,2.0,2.0,1.0,0.0,0.0,NaN,52.328767,...,NaN,NaN,NaN,NaN,"{'neg': 0, 'neu': 0, 'pos': 0, 'compound': 0}",0.000,0.000,0.000,0.0000,23301
3,1700007,95.0,2.0,2.0,1.0,0.0,0.0,0.0,NaN,51.498127,...,NaN,NaN,NaN,NaN,"{'neg': 0, 'neu': 0, 'pos': 0, 'compound': 0}",0.000,0.000,0.000,0.0000,23301
4,1700009,94.0,4.0,2.0,1.0,0.0,0.0,0.0,NaN,51.063829,...,NaN,NaN,NaN,NaN,"{'neg': 0, 'neu': 0, 'pos': 0, 'compound': 0}",0.000,0.000,0.000,0.0000,23400
5,1700011,92.0,4.0,3.0,0.0,0.0,0.0,0.0,1.0,50.367647,...,NaN,NaN,NaN,NaN,"{'neg': 0, 'neu': 0, 'pos': 0, 'compound': 0}",0.000,0.000,0.000,0.0000,23400
6,1700012,91.0,7.0,1.0,0.0,0.0,0.0,0.0,1.0,49.047619,...,NaN,NaN,NaN,NaN,"{'neg': 0, 'neu': 0, 'pos': 0, 'compound': 0}",0.000,0.000,0.000,0.0000,23500
7,1700013,95.0,2.0,2.0,0.0,0.0,0.0,0.0,1.0,52.898550,...,NaN,NaN,NaN,NaN,"{'neg': 0.019, 'neu': 0.737, 'pos': 0.245, 'co...",0.019,0.737,0.245,0.9945,14801
8,1700014,96.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,50.246305,...,NaN,NaN,NaN,NaN,"{'neg': 0, 'neu': 0, 'pos': 0, 'compound': 0}",0.000,0.000,0.000,0.0000,14801
9,1700015,100.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,54.878048,...,NaN,NaN,NaN,NaN,"{'neg': 0.074, 'neu': 0.823, 'pos': 0.103, 'co...",0.074,0.823,0.103,0.9181,14801


In [346]:
schools_df['CZ_ID'].unique()

array(['23200', '23301', '23400', '23500', '14801', '24300', '24900',
       '25602', '24400', '25601', '25401', '23302', '23700', '25500',
       '24200', '23801', '23900', '14500', '24801', '24802', '23600',
       '24701', '14700', '25000', '14802'], dtype=object)

In [347]:
# Define aggregation functions for different data types
aggregation_rules = {}

# Loop through all columns and assign appropriate aggregation
for col in schools_df.columns:
    if col == 'CZ_ID':  
        continue  # Skip CZ_ID since it's the grouping key
    elif schools_df[col].dtype == 'object':  
        aggregation_rules[col] = 'first'  # Keep first non-null value (for categorical/text)
    else:
        aggregation_rules[col] = 'mean'  # Sum numeric values

# Perform aggregation
cz_df = schools_df.groupby('CZ_ID').agg(aggregation_rules).reset_index()



In [348]:
# cz_df = cz_df.fillna(0)  # Replace NaN with 0 


In [349]:
cz_df.head(2)

,CZ_ID,universal-id,demographics_ethnicity_White,demographics_ethnicity_Two or more races,demographics_ethnicity_Hispanic,demographics_ethnicity_Black,demographics_ethnicity_Asian or Pacific Islander,demographics_ethnicity_Native American,demographics_ethnicity_Native Hawaiian or Other Pacific Islander,demographics_ethnicity_Unspecified,...,college_success_Graduates pursuing college_state_value,test_scores_Social Studies_Grade 10_school_value,test_scores_Social Studies_Grade 10_state_value,college_success_Graduates pursuing 4-year college_school_value,college_success_Graduates pursuing 4-year college_state_value,sentiment_scores,neg,neu,pos,compound
0,14500,1.798590e+06,60.745694,5.608122,12.607505,18.037717,3.036224,0.110995,0.086316,1.060086,...,None,NaN,NaN,None,None,"{'neg': 0, 'neu': 0, 'pos': 0, 'compound': 0}",0.028417,0.371346,0.094959,0.369339
1,14700,1.796986e+06,86.267176,4.503817,4.885496,2.717557,1.250000,0.076336,0.152672,1.076923,...,None,NaN,NaN,None,None,"{'neg': 0, 'neu': 0, 'pos': 0, 'compound': 0}",0.023828,0.304795,0.095192,0.327638


In [350]:
cz_df.drop(columns=['universal-id', 'sentiment_scores'], inplace=True)
cz_df.head(2)

,CZ_ID,demographics_ethnicity_White,demographics_ethnicity_Two or more races,demographics_ethnicity_Hispanic,demographics_ethnicity_Black,demographics_ethnicity_Asian or Pacific Islander,demographics_ethnicity_Native American,demographics_ethnicity_Native Hawaiian or Other Pacific Islander,demographics_ethnicity_Unspecified,demographics_gender_Male,...,college_success_Graduates pursuing college_school_value,college_success_Graduates pursuing college_state_value,test_scores_Social Studies_Grade 10_school_value,test_scores_Social Studies_Grade 10_state_value,college_success_Graduates pursuing 4-year college_school_value,college_success_Graduates pursuing 4-year college_state_value,neg,neu,pos,compound
0,14500,60.745694,5.608122,12.607505,18.037717,3.036224,0.110995,0.086316,1.060086,50.330980,...,None,None,NaN,NaN,None,None,0.028417,0.371346,0.094959,0.369339
1,14700,86.267176,4.503817,4.885496,2.717557,1.250000,0.076336,0.152672,1.076923,50.377497,...,None,None,NaN,NaN,None,None,0.023828,0.304795,0.095192,0.327638


In [351]:
cz_df = cz_df.merge(mobility_cz[['CZ_ID', 'Absolute_Upward_Mobility']], on='CZ_ID', how='left')
cz_df.head(2)


,CZ_ID,demographics_ethnicity_White,demographics_ethnicity_Two or more races,demographics_ethnicity_Hispanic,demographics_ethnicity_Black,demographics_ethnicity_Asian or Pacific Islander,demographics_ethnicity_Native American,demographics_ethnicity_Native Hawaiian or Other Pacific Islander,demographics_ethnicity_Unspecified,demographics_gender_Male,...,college_success_Graduates pursuing college_state_value,test_scores_Social Studies_Grade 10_school_value,test_scores_Social Studies_Grade 10_state_value,college_success_Graduates pursuing 4-year college_school_value,college_success_Graduates pursuing 4-year college_state_value,neg,neu,pos,compound,Absolute_Upward_Mobility
0,14500,60.745694,5.608122,12.607505,18.037717,3.036224,0.110995,0.086316,1.060086,50.330980,...,None,NaN,NaN,None,None,0.028417,0.371346,0.094959,0.369339,43.1
1,14700,86.267176,4.503817,4.885496,2.717557,1.250000,0.076336,0.152672,1.076923,50.377497,...,None,NaN,NaN,None,None,0.023828,0.304795,0.095192,0.327638,43.6


In [352]:
cz_df.shape

(25, 174)

In [353]:
cz_df.to_csv("/Users/huangyiyang/Desktop/Winter_2025/MACS30122/final-project-ascending(feb 26)/data/cz_edu_mob.csv", index=False)

**Aggregate to county Level**

In [354]:
school_df2 = schools_df.merge(
    school_tract_cz[['universal-id', 'County_FIPS']],
    on='universal-id',
    how='left'
)
school_df2.head(2)

,universal-id,demographics_ethnicity_White,demographics_ethnicity_Two or more races,demographics_ethnicity_Hispanic,demographics_ethnicity_Black,demographics_ethnicity_Asian or Pacific Islander,demographics_ethnicity_Native American,demographics_ethnicity_Native Hawaiian or Other Pacific Islander,demographics_ethnicity_Unspecified,demographics_gender_Male,...,test_scores_Social Studies_Grade 10_state_value,college_success_Graduates pursuing 4-year college_school_value,college_success_Graduates pursuing 4-year college_state_value,sentiment_scores,neg,neu,pos,compound,CZ_ID,County_FIPS
0,1700001,89.0,1.0,9.0,1.0,0.0,0.0,0.0,NaN,47.407407,...,NaN,NaN,NaN,"{'neg': 0, 'neu': 0, 'pos': 0, 'compound': 0}",0.0,0.0,0.0,0.0,23200,17085
1,1700002,90.0,2.0,9.0,0.0,0.0,0.0,0.0,NaN,49.523809,...,NaN,NaN,NaN,"{'neg': 0, 'neu': 0, 'pos': 0, 'compound': 0}",0.0,0.0,0.0,0.0,23200,17085


In [355]:
len(school_df2['County_FIPS'].unique())

102

In [356]:
# Define aggregation functions for different data types
aggregation_rules = {}

# Loop through all columns and assign appropriate aggregation
for col in school_df2.columns:
    if col == 'County_FIPS':  
        continue  # Skip CZ_ID since it's the grouping key
    elif school_df2[col].dtype == 'object':  
        aggregation_rules[col] = 'first'  # Keep first non-null value (for categorical/text)
    else:
        aggregation_rules[col] = 'mean'  # Sum numeric values

# Perform aggregation
county_df = school_df2.groupby('County_FIPS').agg(aggregation_rules).reset_index()


In [357]:
county_df.head(2)

,County_FIPS,universal-id,demographics_ethnicity_White,demographics_ethnicity_Two or more races,demographics_ethnicity_Hispanic,demographics_ethnicity_Black,demographics_ethnicity_Asian or Pacific Islander,demographics_ethnicity_Native American,demographics_ethnicity_Native Hawaiian or Other Pacific Islander,demographics_ethnicity_Unspecified,...,test_scores_Social Studies_Grade 10_school_value,test_scores_Social Studies_Grade 10_state_value,college_success_Graduates pursuing 4-year college_school_value,college_success_Graduates pursuing 4-year college_state_value,sentiment_scores,neg,neu,pos,compound,CZ_ID
0,17001,1.704610e+06,84.969697,6.303030,2.484848,5.757576,0.387097,0.129032,0.032258,1.0,...,NaN,NaN,None,None,"{'neg': 0, 'neu': 0, 'pos': 0, 'compound': 0}",0.033486,0.447946,0.113189,0.350627,25000
1,17003,1.704606e+06,59.166667,8.333333,1.333333,31.333333,0.000000,0.000000,0.000000,NaN,...,NaN,NaN,None,None,"{'neg': 0, 'neu': 0, 'pos': 0, 'compound': 0}",0.007625,0.085500,0.031875,0.110088,25601


In [358]:
county_df.drop(columns=['universal-id', 'sentiment_scores'], inplace=True)
county_df.head(2)

,County_FIPS,demographics_ethnicity_White,demographics_ethnicity_Two or more races,demographics_ethnicity_Hispanic,demographics_ethnicity_Black,demographics_ethnicity_Asian or Pacific Islander,demographics_ethnicity_Native American,demographics_ethnicity_Native Hawaiian or Other Pacific Islander,demographics_ethnicity_Unspecified,demographics_gender_Male,...,college_success_Graduates pursuing college_state_value,test_scores_Social Studies_Grade 10_school_value,test_scores_Social Studies_Grade 10_state_value,college_success_Graduates pursuing 4-year college_school_value,college_success_Graduates pursuing 4-year college_state_value,neg,neu,pos,compound,CZ_ID
0,17001,84.969697,6.303030,2.484848,5.757576,0.387097,0.129032,0.032258,1.0,52.003004,...,None,NaN,NaN,None,None,0.033486,0.447946,0.113189,0.350627,25000
1,17003,59.166667,8.333333,1.333333,31.333333,0.000000,0.000000,0.000000,NaN,47.417168,...,None,NaN,NaN,None,None,0.007625,0.085500,0.031875,0.110088,25601


In [359]:
county_df = county_df.merge(mobility_county[['County_FIPS', 'Absolute_Upward_Mobility']], on='County_FIPS', how='left')
county_df.head(2)

,County_FIPS,demographics_ethnicity_White,demographics_ethnicity_Two or more races,demographics_ethnicity_Hispanic,demographics_ethnicity_Black,demographics_ethnicity_Asian or Pacific Islander,demographics_ethnicity_Native American,demographics_ethnicity_Native Hawaiian or Other Pacific Islander,demographics_ethnicity_Unspecified,demographics_gender_Male,...,test_scores_Social Studies_Grade 10_school_value,test_scores_Social Studies_Grade 10_state_value,college_success_Graduates pursuing 4-year college_school_value,college_success_Graduates pursuing 4-year college_state_value,neg,neu,pos,compound,CZ_ID,Absolute_Upward_Mobility
0,17001,84.969697,6.303030,2.484848,5.757576,0.387097,0.129032,0.032258,1.0,52.003004,...,NaN,NaN,None,None,0.033486,0.447946,0.113189,0.350627,25000,45.4
1,17003,59.166667,8.333333,1.333333,31.333333,0.000000,0.000000,0.000000,NaN,47.417168,...,NaN,NaN,None,None,0.007625,0.085500,0.031875,0.110088,25601,35.7


In [360]:
county_df.shape

(102, 175)

In [361]:
county_df.to_csv("/Users/huangyiyang/Desktop/Winter_2025/MACS30122/final-project-ascending(feb 26)/data/county_edu_mob.csv", index=False)